In [1]:
import numpy as np
import tensorflow as tf
import chess
import pandas as pd
import io

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, Lambda
from keras import backend as K
from tensorflow import keras

from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Helper Functions~~~

In [0]:
NUM_OUTPUTS = 5


def get_mat(board, piece, color):
  squareSet = board.pieces(piece, color)
  return get_bin_mat(squareSet);

def get_bin_mat(squareSet):
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in squareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
            
  return mat


def get_dual_bin_mat(piece):
  whiteSquareSet = board.pieces(piece, chess.WHITE)
  blackSquareSet = board.pieces(piece, chess.BLACK)
  mat = np.zeros((8,8))
  for x in range(0, 8):
    for y in range(0, 8):
        if(y*8 + x in whiteSquareSet):
          # The 7 - is used to flip the board representation
          mat[7 - y, x] = 1
        elif(y*8 + x in blackSquareSet):
          mat[7 - y, x] = -1
            
  return mat


def interpret_pred(p):
  
  moves = []
  for out in p:
    # There's an extra list wrapping for some reason
    out = out[0]
    start = np.argmax(out[:64])
    end = np.argmax(out[64:])
       
    moves.append((chess.SQUARE_NAMES[start],
                  chess.SQUARE_NAMES[end]))
    
  return moves

  

def create_input(board):
  if(board.turn):
    turnMat = np.ones((8,8))
  else:
    turnMat = np.negative(np.ones((8,8)))
  mat = np.array(([get_mat(board, i, chess.WHITE) for i in range(1,7)] + [get_mat(board, i, chess.BLACK) for i in range(1,7)] + [turnMat]))
  y = mat.reshape((-1, 13, 8, 8))
  return y

def create_output(moves):
  output = []
  for i in range(0, NUM_OUTPUTS):
    startPos = moves[i][0]
    endPos = moves[i][1]
    
    mat = np.zeros(128)
    
    mat[startPos] = 1
    mat[64 + endPos] = 1
        
    output.append(mat)
  
  return output

def create_training_set(df):
  inputs = []
  outputs = []

  for i in range(len(df)):
    pos = df.iloc[i].Position
    moves = eval(df.iloc[i].Moves)

    board.set_fen(pos)
    if i > 0:
      inputs = np.append(inputs, create_input(board), axis=0)
    else:
      inputs = create_input(board)
    #outputs.append(create_output(moves))
    
  for j in range(0, NUM_OUTPUTS):
    output = []
    for k in range(len(df)):
      moves = eval(df.iloc[k].Moves)

      startPos = moves[j][0]
      endPos = moves[j][1]
    
      mat = np.zeros(128)

      mat[startPos] = 1
      mat[64 + endPos] = 1
        
      output.append(mat)
    
    outputs.append(output)

  return (inputs, outputs)

In [3]:
np.zeros(128).shape

(128,)

Input data:

  (6 pieces) * 2 players + 1 for turn
  
  8 rows
  
  8 columns
  
 => 8 * 8 * 13 = 832
 
 
 Output data:
 
 - Assume predicting five moves
 - initial location, and new location
 
 
Array of objects/tuples
[{initial location, new location},...]
 
 
 => 5 * 2 * 64 = 640
  

In [4]:
inp = Input((13,8,8))


x = Conv2D(128, kernel_size=3, activation='relu', input_shape=(13,8,8))(inp)

x = Conv2D(64, kernel_size=3, activation='relu', input_shape=(13,8,8))(x)

x = Conv2D(32, kernel_size=3, activation='relu', input_shape=(13,8,8))(x)


x = Flatten()(x)


outputs = []

for n in range(NUM_OUTPUTS):
  outputs.append(Dense(128, activation='softmax')(x))

model = Model(inp, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

model.summary()

W0807 00:16:14.320337 140527518738304 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13, 8, 8)]   0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 11, 6, 128)   9344        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 9, 4, 64)     73792       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 2, 32)     18464       conv2d_1[0][0]                   
______________________________________________________________________________________________

In [5]:
board = chess.Board()

y = create_input(board)

p = model.predict(y)

interpret_pred(p)

[('a6', 'h1'), ('e1', 'g2'), ('f3', 'h2'), ('d5', 'f7'), ('c4', 'c2')]

In [6]:
df=pd.read_csv('drive/My Drive/Chess/train.csv')

df.head()

,Position,Moves
0,rnbqkbnr/ppppppp1/7p/8/4P3/2N5/PPPP1PPP/R1BQKB...,"[(62, 45), (52, 36), (49, 41), (51, 43), (50, ..."
1,rnbqkbnr/pp1pppp1/7p/2p5/4P3/2N5/PPPP1PPP/R1BQ...,"[(18, 1), (18, 33), (5, 33), (13, 29), (0, 1)]"
2,rnbqkbnr/pp1pppp1/7p/2p5/4P3/8/PPPP1PPP/RNBQKB...,"[(54, 38), (53, 45), (57, 42), (52, 36), (59, ..."
3,rnbqkbnr/pp1ppp2/7p/2p3p1/4P3/8/PPPP1PPP/RNBQK...,"[(6, 23), (10, 26), (3, 12), (10, 18), (28, 36)]"
4,rnbqkbnr/pp1ppp2/7p/2p3p1/4P3/3B4/PPPP1PPP/RNB...,"[(48, 32), (57, 42), (59, 32), (34, 26), (59, ..."


In [7]:
df = df[:10000] # Set an initial limit

SPLIT_SIZE = 1000
NUM_CHUNKS = len(df)/SPLIT_SIZE

i = 1
for chunk in np.array_split(df, NUM_CHUNKS):
  inputs, outputs = create_training_set(chunk)
  model.fit(inputs, outputs, epochs=5)
  print('Model fit for ' + str(i * SPLIT_SIZE) + 'training samples.')
  i = i + 1

Epoch 1/5
1000/1000 [==============================] - 1s 556us/sample - loss: 48.6991 - dense_loss: 9.7539 - dense_1_loss: 9.7176 - dense_2_loss: 9.7575 - dense_3_loss: 9.7444 - dense_4_loss: 9.7343 - dense_acc: 0.0120 - dense_1_acc: 0.0080 - dense_2_acc: 0.0010 - dense_3_acc: 0.0100 - dense_4_acc: 0.0000e+00
Epoch 2/5
1000/1000 [==============================] - 0s 264us/sample - loss: 112.7897 - dense_loss: 23.3339 - dense_1_loss: 22.7503 - dense_2_loss: 24.3027 - dense_3_loss: 24.1800 - dense_4_loss: 23.1238 - dense_acc: 0.0090 - dense_1_acc: 0.0120 - dense_2_acc: 0.0240 - dense_3_acc: 0.0180 - dense_4_acc: 0.0220
Epoch 3/5
1000/1000 [==============================] - 0s 267us/sample - loss: 3490.7357 - dense_loss: 696.3323 - dense_1_loss: 741.9869 - dense_2_loss: 735.4409 - dense_3_loss: 734.2175 - dense_4_loss: 759.6875 - dense_acc: 0.0160 - dense_1_acc: 0.0100 - dense_2_acc: 0.0130 - dense_3_acc: 0.0170 - dense_4_acc: 0.0150
Epoch 4/5
1000/1000 [==============================] -

In [8]:
board = chess.Board()

y = create_input(board)

p = model.predict(y)

interpret_pred(p)

[('a1', 'f7'), ('h2', 'a1'), ('e5', 'a1'), ('e8', 'a1'), ('b2', 'a1')]